In [ ]:
!pip install langchain langchain-community langchain-core

In [ ]:
!pip install -U langchain-ollama

In [ ]:
!pip install -U sentence-transformers

In [ ]:
!pip install aift

# mistal model

In [10]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_ollama import embeddings
from langchain_ollama import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import CharacterTextSplitter

model_local = ChatOllama(model="mistral")

# 1. Split data into chunks
urls = [
    "https://ollama.com/",
    "https://ollama.com/blog/windows-preview",
    "https://ollama.com/blog/openai-compatibility",
]
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=7500, chunk_overlap=100)
doc_splits = text_splitter.split_documents(docs_list)

# 2. Convert documents to Embeddings and store them
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=embeddings.OllamaEmbeddings(model='nomic-embed-text'),
)
retriever = vectorstore.as_retriever()

# 3. Before RAG
print("Before RAG\n")
before_rag_template = "What is {topic}"
before_rag_prompt = ChatPromptTemplate.from_template(before_rag_template)
before_rag_chain = before_rag_prompt | model_local | StrOutputParser()
print(before_rag_chain.invoke({"topic": "Ollama"}))

# 4. After RAG
print("\n########\nAfter RAG\n")
after_rag_template = """Answer the question based only on the following context:
{context}
Question: {question}
"""
after_rag_prompt = ChatPromptTemplate.from_template(after_rag_template)
after_rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | after_rag_prompt
    | model_local
    | StrOutputParser()
)
print(after_rag_chain.invoke("What is Ollama?"))

# loader = PyPDFLoader("Ollama.pdf")
# doc_splits = loader.load_and_split()

Before RAG

 Ollama is a large language model developed by Mistral AI, a company based in Paris, France. As of my last update, it was announced but not yet publicly released. The model is said to be a significant advancement in the field of artificial intelligence and could potentially compete with models like ChatGPT from OpenAI or Bard from Google. However, I don't have real-time updates, so please verify from a reliable source for the latest information.

########
After RAG

 Based on the provided text snippets, Ollama appears to be a company or platform that provides large language models. The text mentions various ways to interact with Ollama, such as through its blog, Discord, GitHub, and by downloading its models. The text also suggests that Ollama supports existing tooling built for OpenAI and has an API that can be accessed using PowerShell.


# PATHUMMA LLM + simcse-model-m-bert-thai-cased

In [14]:
from sentence_transformers import SentenceTransformer

class CustomEmbeddings:
    def __init__(self, model_name="mrp/simcse-model-m-bert-thai-cased"):
        """
        Initialize the embedding model using SentenceTransformer.
        :param model_name: Name of the pre-trained model
        """
        self.model = SentenceTransformer(model_name)

    def embed_query(self, text):
        """
        Generate embeddings for a single query.
        :param text: Input text to embed
        :return: Embedding vector as a Python list
        """
        embedding = self.model.encode([text])
        return embedding[0].tolist()  # Convert NumPy array to list

    async def aembed_query(self, text):
        """
        Asynchronous version of `embed_query`.
        :param text: Input text to embed
        :return: Embedding vector as a Python list
        """
        return self.embed_query(text)

    def embed_documents(self, texts):
        """
        Generate embeddings for multiple documents.
        :param texts: List of input texts to embed
        :return: List of embedding vectors as Python lists
        """
        embeddings = self.model.encode(texts)
        return [embedding.tolist() for embedding in embeddings]

    async def aembed_documents(self, texts):
        """
        Asynchronous version of `embed_documents`.
        :param texts: List of input texts to embed
        :return: List of embedding vectors as Python lists
        """
        return self.embed_documents(texts)


In [15]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_ollama import embeddings
from langchain_ollama import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import CharacterTextSplitter


file_path = r"C:\Users\Admin\Desktop\LLM\eyeeieiei.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=7500, chunk_overlap=100)
doc_splits = text_splitter.split_documents(docs)

# 2. Convert documents to Embeddings and store them
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=CustomEmbeddings(model_name="mrp/simcse-model-m-bert-thai-cased"),
)
retriever = vectorstore.as_retriever()

45


In [ ]:
from aift.multimodal import textqa
from aift import setting

# Set Pathumma API Key
setting.set_api_key('API_KEY')

# Define a simple wrapper for Pathumma
class PathummaModel:
    def __init__(self):
        pass

    def generate(self, instruction: str, return_json: bool = False):
        response = textqa.generate(instruction=instruction, return_json=return_json)
        if return_json:
            return response.get("content", "")
        return response

    def __call__(self, input: str):
        return self.generate(input, return_json=False)

# Initialize Pathumma Model
model_local = PathummaModel()

In [20]:
# 3. Before RAG
print("Before RAG\n")
before_rag_template = "ใครเป็นผู้จัดทำ {topic} "
before_rag_prompt = ChatPromptTemplate.from_template(before_rag_template)
before_rag_chain = before_rag_prompt | model_local | StrOutputParser()
print(before_rag_chain.invoke({"topic": "การวิเคราะห์และจำแนกระยะอาการเบาหวานขึ้นตาโดยใช้ Classification Analysis and Classification of Diabetic Retinopathy Stages using Classification"}))

# 4. After RAG
print("\n########\nAfter RAG\n")
after_rag_template = """ตอบคำถามโดยพิจารณาจากบริบทต่อไปนี้เท่านั้น:
{context}
คำถาม: {question}
"""
after_rag_prompt = ChatPromptTemplate.from_template(after_rag_template)

# Query retriever for context and pass to Pathumma
question = "ใครเป็นผู้จัดทำการวิเคราะห์และจำแนกระยะอาการเบาหวานขึ้นตาโดยใช้ Classification Analysis and Classification of Diabetic Retinopathy Stages using Classification"
retrieved_context = retriever.get_relevant_documents(question)
context = "\n".join([doc.page_content for doc in retrieved_context])

after_rag_chain = after_rag_prompt.invoke({
    "context": context,
    "question": question,
})
response = model_local(after_rag_chain)
print("response")
print(response)

Before RAG

การวิเคราะห์และจำแนกระยะอาการเบาหวานขึ้นตาโดยใช้ Classification Analysis and Classification of Diabetic Retinopathy Stages using Classification เป็นผลงานของ ชัชวาลย์ สุริยะพันธุ์ และคณะ จากมหาวิทยาลัยมหิดล

########
After RAG

response
ภัคพล อาจบุราย,ศิระภัทร ไทรพงษ์พันธ์ และศุภชัย สุปัญญา


# PATHUMMA LLM + nomic-embed-text

In [20]:
from aift.multimodal import textqa
from aift import setting

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_ollama import embeddings
from langchain_ollama import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import CharacterTextSplitter



# Set Pathumma API Key
setting.set_api_key('hChDEu87Ln2VThG8HVqEweoeVoVSGHgJ')

# Define a simple wrapper for Pathumma
class PathummaModel:
    def __init__(self):
        pass

    def generate(self, instruction: str, return_json: bool = False):
        response = textqa.generate(instruction=instruction, return_json=return_json)
        if return_json:
            return response.get("content", "")
        return response

    def __call__(self, input: str):
        return self.generate(input, return_json=False)

# Initialize Pathumma Model
model_local = PathummaModel()

# 1. Split data into chunks
urls = [
    "https://th.wikipedia.org/wiki/%E0%B8%AD%E0%B8%B8%E0%B8%97%E0%B8%A2%E0%B8%B2%E0%B8%99%E0%B8%9B%E0%B8%A3%E0%B8%B0%E0%B8%A7%E0%B8%B1%E0%B8%95%E0%B8%B4%E0%B8%A8%E0%B8%B2%E0%B8%AA%E0%B8%95%E0%B8%A3%E0%B9%8C%E0%B8%9E%E0%B8%99%E0%B8%A1%E0%B8%A3%E0%B8%B8%E0%B9%89%E0%B8%87",
   
]
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=7500, chunk_overlap=100)
doc_splits = text_splitter.split_documents(docs_list)

# 2. Convert documents to Embeddings and store them
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="rag-chroma",
    embedding=embeddings.OllamaEmbeddings(model='nomic-embed-text'),
)
retriever = vectorstore.as_retriever()

# 3. Before RAG
print("Before RAG\n")
before_rag_template = "{topic} มีความเป็นมายังไง"
before_rag_prompt = ChatPromptTemplate.from_template(before_rag_template)
before_rag_chain = before_rag_prompt | model_local | StrOutputParser()
print(before_rag_chain.invoke({"topic": "ปราสาทหินพนมรุ้ง"}))

# 4. After RAG
print("\n########\nAfter RAG\n")
after_rag_template = """ตอบคำถามโดยพิจารณาจากบริบทต่อไปนี้เท่านั้น:
{context}
คำถาม: {question}
"""
after_rag_prompt = ChatPromptTemplate.from_template(after_rag_template)

# Query retriever for context and pass to Pathumma
question = "ปราสาทหินพนมรุ้ง มีความเป็นมายังไง"
retrieved_context = retriever.get_relevant_documents(question)
context = "\n".join([doc.page_content for doc in retrieved_context])

after_rag_chain = after_rag_prompt.invoke({
    "context": context,
    "question": question,
})
response = model_local(after_rag_chain)
print(response)


Before RAG

ปราสาทหินพนมรุ้ง ตั้งอยู่ในอำเภอพนมดงรัก จังหวัดบุรีรัมย์ เป็นหนึ่งในปราสาทหินที่ใหญ่และสวยงามที่สุดของประเทศไทย มันถูกสร้างขึ้นโดยอาณาจักรเขมรโบราณ ในศตวรรษที่ 12 และเป็นส่วนหนึ่งของเครือข่ายของปราสาทหินที่สำคัญในภูมิภาคนี้

ประวัติศาสตร์: 
- ปราสาทหินพนมรุ้งสร้างขึ้นในศตวรรษที่ 12 โดยพระเจ้าสุโขทัยที่สอง
- มันเป็นส่วนหนึ่งของเครือข่ายของปราสาทหินที่สำคัญในภูมิภาคนี้ และได้รับอิทธิพลจากศิลปะเขมรโบราณ

สถาปัตยกรรม: 
- โครงสร้างหลักของปราสาทเป็นแบบสี่เหลี่ยมผืนผ้า มีฐานกว้างประมาณ 100 เมตร และสูงประมาณ 30 เมตร
- ด้านบนของปราสาทหินพนมรุ้งมีช่องว่างสำหรับการสวดมนต์และมีภาพวาดฝาผนังที่สวยงาม

ความสำคัญ: 
- ปราสาทหินพนมรุ้งเป็นหนึ่งในสถานที่ทางประวัติศาสตร์ที่สำคัญของประเทศไทย
- มันได้รับการขึ้นทะเบียนเป็นมรดกโลกโดยองค์การยูเนสโก และเป็นแหล่งท่องเที่ยวที่สำคัญในจังหวัดบุรีรัมย์

ความสวยงาม: 
- ปราสาทหินพนมรุ้งโดดเด่นด้วยสถาปัตยกรรมแบบเขมรโบราณและภาพวาดฝาผนังที่สวยงาม
- มันเป็นสถานที่ที่เหมาะสำหรับการสำรวจประวัติศาส

########
After RAG

คำถาม: ปราสาทหินพนมรุ้ง มีความเป็นมายังไง

